# Segmenting and Clustering Neighborhoods in Toronto

### Import necessary libraries

### Scrape Wikipedia page

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

     |████████████████████████████████| 102kB 12.1MB/s ta 0:00:01
     |████████████████████████████████| 5.8MB 16.3MB/s eta 0:00:01


In [20]:
# use bs4 to get the source code from the url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup = BeautifulSoup(source,'lxml')
#print(soup)
table = soup.find('table')
#print(table)

In [21]:
neigh = table.find_all('tr')
#print(neigh)

In [15]:
d = []
for neigh in neigh:
    cols=neigh.find_all('td')
    cols=[x.text.strip() for x in cols]
    d.append(cols)

In [16]:
columns=['PostalCode','Borough','Neighbourhood']
df_TO = pd.DataFrame(data=d,columns=columns)[1:]
print(df_TO.count())
df_TO.head()

PostalCode       288
Borough          288
Neighbourhood    288
dtype: int64


,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [17]:
#Ignore cells with a borough that is Not assigned.
df_TO_clean =df_TO[df_TO['Borough'] !='Not assigned']

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
for index, row in df_TO_clean.iterrows():
    if row['Neighbourhood']=='Not assigned':
        row['Neighbourhood']=row['Borough']


In [18]:
df_TO_clean_gr = df_TO_clean.groupby(['PostalCode','Borough'], sort=False).agg( ','.join)
df_TO_clean_gr = df_TO_clean_gr.reset_index()
df_TO_clean_gr.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [19]:
df_TO_clean_gr.shape

(103, 3)